In [1]:
from scripts import models, graph, coarsening,GCN_Model,DenseGCN_Model

import numpy as np
import pandas as pd
from scipy import sparse
from tensorflow.python.framework import ops
import tensorflow as tf
# Clear all the stack and use GPU resources as much as possible
ops.reset_default_graph()
config= tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)

2024-02-01 19:41:35.594892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


2024-02-01 19:41:41.920760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_data= np.load('training_set.npy').astype('float32')
train_labels= np.load('training_label.npy').astype('float32')
test_data= np.load('test_set.npy').astype('float32')
test_labels= np.load('test_label.npy').astype('float32')
print('==============> Data read!')
test_labels=test_labels.reshape(4051000,)
train_labels=train_labels.reshape(16204000,)

==============> Data read!


In [3]:
Adjacency_Matrix = np.load('Adjacency_Matrix.npy').astype('float32')
Adjacency_Matrix = sparse.csr_matrix(Adjacency_Matrix)
print('==============> Adjancy matrix read!')
graphs, perm = coarsening.coarsen(Adjacency_Matrix, levels=5, self_connections=False)
X_train = coarsening.perm_data(train_data, perm)
X_test  = coarsening.perm_data(test_data,  perm)
print('==============>coarsening done!')

==============> Adjancy matrix read!
Layer 0: M_0 = |V| = 384 nodes (372 added),|E| = 66 edges
Layer 1: M_1 = |V| = 192 nodes (180 added),|E| = 66 edges
Layer 2: M_2 = |V| = 96 nodes (84 added),|E| = 66 edges
Layer 3: M_3 = |V| = 48 nodes (36 added),|E| = 66 edges
Layer 4: M_4 = |V| = 24 nodes (12 added),|E| = 66 edges
Layer 5: M_5 = |V| = 12 nodes (0 added),|E| = 66 edges


In [ ]:
L = [graph.laplacian(Adjacency_Matrix, normalized=True) for Adjacency_Matrix in graphs]
print('==============>laplacian obtained!')
# graph.plot_spectrum(L)

In [ ]:
params = dict()
params['dir_name']       = 'folder1'
params['num_epochs']     = 100
params['batch_size']     = 1024
params['eval_frequency'] = 100
# Building blocks.
params['filter'] = 'chebyshev5'
params['brelu']  = 'b2relu'
params['pool']   = 'mpool1'
# Architecture.
params['F'] = [16, 32, 64, 128, 256, 512]  # Number of graph convolutional filters.
params['K'] = [2, 2, 2, 2, 2, 2]           # Polynomial orders.
params['p'] = [2, 2, 2, 2, 2, 2]           # Pooling sizes.
params['M'] = [2]                          # Output dimensionality of fully connected layers.
# Optimization.
params['regularization'] = 0.000001  # L2 regularization
params['dropout']        = 0.50      # Dropout rate
params['learning_rate']  = 0.000001  # Learning rate
params['decay_rate']     = 1         # Learning rate Decay == 1 means no Decay
params['momentum']       = 0         # momentum == 0 means Use Adam Optimizer
params['decay_steps']    = np.shape(train_data)[0] / params['batch_size']
print('==============>parameters  selected!')

In [ ]:
model = models.cgcnn(L, **params)
#model = GCN_Model.cgcnn(L, **params)
#model=DenseGCN_Model.cgcnn(L, **params)
print('==============>model  established!')
accuracy, loss, t_step = model.fit(X_train, train_labels, X_test, test_labels)
print('==============>model  fitted!')